---

# **Top_5_Rated_Tourist_Places_Recommending_Agent**

---

In [ ]:
!pip install langchain langchain_community openai

In [ ]:

import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from PIL import Image
from io import BytesIO

# Set your API keys
os.environ["OPENAI_API_KEY"] = "##########"
google_maps_api_key = "#########"  # Replace with your Google Static Maps API key
google_places_api_key = google_maps_api_key  # Using the same API key for both Places and Maps

# Function to get latitude and longitude from location name using Google Geocoding API
def get_lat_lng(location_name):
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location_name}&key={google_maps_api_key}"

    # Make the request to the Google Geocoding API
    response = requests.get(geocode_url)

    if response.status_code == 200:
        geo_data = response.json()
        if geo_data['results']:
            location = geo_data['results'][0]['geometry']['location']
            lat_lng = f"{location['lat']},{location['lng']}"
            return lat_lng
        else:
            return "Location not found."
    else:
        return "Error fetching location data."

# Function to fetch nearby tourist places using Google Places API
def get_tourist_places(location_name):
    # Get the latitude and longitude from the location name
    lat_lng = get_lat_lng(location_name)

    if "Error" in lat_lng or "Location not found" in lat_lng:
        return lat_lng

    # Define the Google Places API URL for nearby search
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Set the parameters for the nearby search
    params = {
        "location": lat_lng,  # Latitude and longitude of the location
        "radius": 5000,        # Search radius in meters (5km)
        "type": "tourist_attraction",  # We're interested in tourist attractions
        "key": google_places_api_key   # Your Google Places API key
    }

    # Make the request to the Places API
    response = requests.get(url, params=params)

    if response.status_code == 200:
        places = response.json().get('results', [])
        # Format the results to return a readable list of tourist places
        if places:
            place_names = [f"{place['name']} - Rating: {place.get('rating', 'N/A')}" for place in places[:5]]
            return "\n".join(place_names)  # Return the top 5 tourist places
        else:
            return "No tourist attractions found nearby."
    else:
        return f"Could not retrieve tourist places for {location_name}. Please try another location."

# Define the tourist places tool
tourist_places_tool = Tool(
    name="google_places",
    func=get_tourist_places,
    description="Provides a list of nearby tourist places for a given location."
)

# Initialize the language model (OpenAI GPT-4 or GPT-3.5)
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Set up memory for personalized conversations
memory = ConversationBufferMemory()

# Create the agent using the latest LangChain methods
agent = initialize_agent(
    tools=[tourist_places_tool],  # Add tools for maps and tourist places
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# Example user interaction
# User asks for nearby tourist places for a specific location
#user_query = "Paris"  # You can change this to any location
response_places = agent.run(user_query)
print(response_places)
